In [ ]:
import warnings
warnings.filterwarnings('ignore')

!nvidia-smi

Thu Oct  3 07:55:50 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0              45W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
import torch
print(f"Torch + Cuda Version: {torch.__version__}")
print(f"Cuda Version Requied: {torch.version.cuda}")
print(f"Cuda Available: {torch.cuda.is_available()}")
print(f"Cuda Device Count: {torch.cuda.device_count()}")
print(f"Cuda Device Name: {torch.cuda.get_device_name(0)}")
!python --version

Torch + Cuda Version: 2.4.1+cu121
Cuda Version Requied: 12.1
Cuda Available: True
Cuda Device Count: 1
Cuda Device Name: NVIDIA A100-SXM4-40GB
Python 3.10.12


In [ ]:
!git clone https://github.com/buaacyw/MeshAnythingV2

Cloning into 'MeshAnythingV2'...
remote: Enumerating objects: 156, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 156 (delta 22), reused 19 (delta 19), pack-reused 120 (from 1)
Receiving objects: 100% (156/156), 15.42 MiB | 16.79 MiB/s, done.
Resolving deltas: 100% (50/50), done.


In [ ]:
%cd MeshAnythingV2
!ls

/content/MeshAnythingV2
adjacent_mesh_tokenization.py  demo	 gt_examples  main.py	    mesh_to_pc.py  README.md
app.py			       examples  LICENSE.txt  MeshAnything  pc_examples    requirements.txt


In [ ]:
!pip install flash-attn --no-build-isolation -q
!pip install -r requirements.txt -q
!pip install open3d -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 64.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 690.8/690.8 kB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.9/409.9 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.

In [24]:
import os

sample = 'fd559c38655afa5052a68d3513597b6da121cef0'

DATA_DIR = os.path.join("/content/point_clouds/", sample)
OUTPUT_DIR = os.path.join(DATA_DIR, "output")
DSM_PATH = os.path.join(DATA_DIR, 'dsm.json')
NPY_PATH = os.path.join(DATA_DIR, 'dsm_and_normals.npy')

In [25]:
import json
import numpy as np
import open3d as o3d

# Step 1: Load the JSON file containing the XYZ coordinates
def load_json(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)
    return np.array(data)

# Step 2: Compute normals using Open3D
def compute_normals(xyz_points):
    # Create an Open3D PointCloud object
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(xyz_points)

    # Estimate the normals
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

    # Retrieve the normals
    normals = np.asarray(pcd.normals)
    return normals

# Step 3: Save as .npy in the (N, 6) pc_normal format
def save_as_npy(xyz_points, normals, npy_file):
    # Combine the XYZ coordinates with their corresponding normals
    pc_normal_data = np.hstack((xyz_points, normals))

    np.save(npy_file, pc_normal_data)
    print(f"Point cloud with normals saved to {npy_file}")


xyz_points = load_json(DSM_PATH)
normals = compute_normals(xyz_points)
save_as_npy(xyz_points, normals, NPY_PATH)

Point cloud with normals saved to /content/point_clouds/fd559c38655afa5052a68d3513597b6da121cef0/dsm_and_normals.npy


In [27]:
# inference for single file
!python main.py --input_path {NPY_PATH} --out_dir {OUTPUT_DIR} --input_type pc_normal

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:463: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/content/MeshAnythingV2/MeshAnything/miche/michelangelo/models/modules/checkpoint.py:41: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, run_function, length, *args):
/content/MeshAnythingV2/MeshAnything/miche/michelangelo/models/modules/checkpoint.py:52: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, *output_grads):
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you